In [2]:
import numpy as np
import pandas as pd
import re
from rdkit import Chem
from rdkit.Chem import Descriptors

In [29]:
# 读取数据
df = pd.read_csv('data_filter.csv', encoding='utf-8-sig')
print(df.head(898))

          ID                  NAME  \
0    10001.0  2-mercaptopyrimidine   
1    10002.0  2-mercaptopyrimidine   
2    10003.0  2-mercaptopyrimidine   
3    10004.0  2-mercaptopyrimidine   
4    10005.0  2-mercaptopyrimidine   
..       ...                   ...   
893  10911.0   Monoethylene glycol   
894  10912.0   Monoethylene glycol   
895  10913.0         Thiourea (TU)   
896  10914.0         berberine（BB）   
897  10915.0         berberine（BB）   

                                                SMILES  NaClg/L  Temp  \
0                                        S=c1nccc[nH]1     10.0  313K   
1                                        S=c1nccc[nH]1     50.0  313K   
2                                        S=c1nccc[nH]1    100.0  313K   
3                                        S=c1nccc[nH]1    150.0  313K   
4                                        S=c1nccc[nH]1    200.0  313K   
..                                                 ...      ...   ...   
893  C=C(c1ccccc1)C5(c2ccccc2)/C

In [4]:
for i,item in zip(df['ID'],df['Concentration']):
    item = item.replace(' ','').replace('\n','')
    item = item.replace('×', '*').replace('μ', 'mu').replace('wt.%', 'wt%').replace('ppm','PPM').replace('／','/')
    item = item.replace('?*?','*').replace('*?','*').replace('?5?','-5')
    #item = item.replace('ï¼','/').replace('dm?3', '/l').replace('moll?1','mol/l').replace('L?1','/l').replace('l?1','/l').replace('Â·L-1','/l')
    item = item.replace('?','-')
    item = item.replace('moldm-3','M').replace('moll-1','M').replace('mgL-1','mg/L').replace('mol·L-1','M').replace('mg-L-1','mg/L')
    item = item.replace('Mol/L','M').replace('M/L','M').replace('MOL/L','M').replace('mol/L','M').replace('mol/l','M').replace('mol/m3','mM')
    print(item)

20PPM
20PPM
20PPM
20PPM
20PPM
100PPM
7*10-5M
15*10-5M
35*10-5M
70*10-5M
7*10-5M
15*10-5M
35*10-5M
70*10-5M
1*10-5M
2.5*10-5M
5*10-5M
1*10-4M
5*10-4M
1*10-5M
2.5*10-5M
5*10-5M
1*10-4M
5*10-4M
1*10-5M
2.5*10-5M
5*10-5M
1*10-4M
5*10-4M
1*10-5M
2.5*10-5M
5*10-5M
1*10-4M
5*10-4M
25PPM
50PPM
75PPM
100PPM
10PPM
25PPM
50PPM
100PPM
5PPM
10PPM
25PPM
50PPM
100PPM
25PPM
50PPM
75PPM
100PPM
150PPM
25PPM
50PPM
75PPM
100PPM
150PPM
25PPM
50PPM
75PPM
100PPM
150PPM
100PPM
100PPM
100PPM
100PPM
100PPM
100PPM
10PPM
25PPM
50PPM
75PPM
100PPM
25PPM
50PPM
75PPM
100PPM
150PPM
200PPM
2*10-4M
2*10-4M
2*10-4M
60PPM
60PPM
50PPM
100PPM
200PPM
400PPM
50PPM
100PPM
200PPM
400PPM
1PPM
2.5PPM
5PPM
7.5PPM
10PPM
1PPM
2.5PPM
5PPM
7.5PPM
10PPM
0.1mM
0.2mM
0.4mM
0.8mM
0.1mM
0.2mM
0.4mM
0.8mM
0.0125wt%
0.025wt%
0.05wt%
0.075wt%
0.1wt%
5PPM
10PPM
50PPM
5PPM
10PPM
50PPM
1mM
3mM
1mM
3mM
1*10-6M
5*10-6M
1*10-5M
5*10-5M
1*10-4M
5*10-4M
1*10-6M
5*10-6M
1*10-5M
5*10-5M
1*10-4M
5*10-4M
1*10-6M
5*10-6M
1*10-5M
5*10-5M
1*10-4M
5*10-4M
10

In [30]:
#将浓度列和温度列的符号去掉
df['Concentration'] = df['Concentration'].str.replace(' ','').str.replace('\n','')
df['Concentration'] = df['Concentration'].str.replace('×', '*').str.replace('μ', 'mu').str.replace('wt.%', 'wt%').str.replace('ppm','PPM').str.replace('／','/')
df['Concentration'] = df['Concentration'].str.replace('?*?','*').str.replace('*?','*').str.replace('?5?','-5')
df['Concentration'] = df['Concentration'].str.replace('?','-')
df['Concentration'] = df['Concentration'].str.replace('moldm-3','M').str.replace('moll-1','M').str.replace('mgL-1','mg/L').str.replace('mol·L-1','M').str.replace('mg-L-1','mg/L')
df['Concentration'] = df['Concentration'].str.replace('Mol/L','M').str.replace('M/L','M').str.replace('MOL/L','M').str.replace('mol/L','M').str.replace('mol/l','M').str.replace('mol/m3','mM')
df['TempK'] = df['Temp'].str.replace('room temperature', '298').str.replace('±1','').str.replace(' ','').str.replace('K','').str.replace('k','')
df['IE'] = df['IE'].str.rstrip('%').astype('float') / 100
df['TempK'] = df['TempK'].fillna('298')
df.tail(20)

,ID,NAME,SMILES,NaClg/L,Temp,Concentration,IE,TempK
1662,11703.0,N-benzyl chitosan oligosaccharide quaternary a...,C[N+](C)(C)CC(O)CNC1C(O)CC(CO)OC1OC4OC(CO)C(OC...,35.0000,353K,100mg/L,0.8806,353.0
1663,11704.0,N-propyl chitosan oligosaccharide quaternary ...,CCC=NC3C(O)OC(CO)C(OC2C(O)C(N)C(OC1OC(CO)CC(O)...,35.0000,353K,30mg/L,0.7152,353.0
1664,11705.0,N-propyl chitosan oligosaccharide quaternary ...,CCC=NC3C(O)OC(CO)C(OC2C(O)C(N)C(OC1OC(CO)CC(O)...,35.0000,353K,50mg/L,0.8773,353.0
1665,11706.0,N-propyl chitosan oligosaccharide quaternary ...,CCC=NC3C(O)OC(CO)C(OC2C(O)C(N)C(OC1OC(CO)CC(O)...,35.0000,353K,80mg/L,0.8343,353.0
1666,11707.0,N-propyl chitosan oligosaccharide quaternary ...,CCC=NC3C(O)OC(CO)C(OC2C(O)C(N)C(OC1OC(CO)CC(O)...,35.0000,353K,100mg/L,0.9190,353.0
1667,11708.0,4-Carboxyphenylboronic acid (CPBA),O=C(O)c1ccc(B(O)O)cc1,0.0585,298K,0.12mM,0.7713,298.0
1668,11709.0,4-Carboxyphenylboronic acid (CPBA),O=C(O)c1ccc(B(O)O)cc1,0.0585,298K,0.60mM,0.9267,298.0
1669,11710.0,4-Carboxyphenylboronic acid (CPBA),O=C(O)c1ccc(B(O)O)cc1,0.0585,298K,2.41mM,0.9536,298.0
1670,11711.0,β-amino acids（GZC-8）,CCCCCCCC[NH2+]CCC(=O)[O-],0.0585,343K,25PPM,0.6147,343.0
1671,11712.0,β-amino acids（GZC-18）,CCCCCCCCCCCCCCCCCC[NH2+]CCC(=O)[O-],0.0585,343K,25PPM,0.9014,343.0


In [37]:
#温度列转化为int
for item in df['TempK']:
    item = int(item)
df

,ID,NAME,SMILES,NaClg/L,Temp,Concentration,IE,TempK,Fixed_Concentration,Concentrationg/L
0,10001.0,2-mercaptopyrimidine,S=c1nccc[nH]1,10.0,313K,20PPM,0.9124,313.0,20PPM,0.02
1,10002.0,2-mercaptopyrimidine,S=c1nccc[nH]1,50.0,313K,20PPM,0.8556,313.0,20PPM,0.02
2,10003.0,2-mercaptopyrimidine,S=c1nccc[nH]1,100.0,313K,20PPM,0.8331,313.0,20PPM,0.02
3,10004.0,2-mercaptopyrimidine,S=c1nccc[nH]1,150.0,313K,20PPM,0.8173,313.0,20PPM,0.02
4,10005.0,2-mercaptopyrimidine,S=c1nccc[nH]1,200.0,313K,20PPM,0.7629,313.0,20PPM,0.02
...,...,...,...,...,...,...,...,...,...,...
1677,11718.0,2-undecyl-1-ethylamino imidazoline (2UEI),CCCCCCCCCCC/C1=N/CCN1CCN,35.0,298K,80mg/L,0.7523,298.0,80mg/L,0.08
1678,11719.0,2-undecyl-1-ethylamino imidazoline (2UEI),CCCCCCCCCCC/C1=N/CCN1CCN,35.0,298K,100mg/L,0.8767,298.0,100mg/L,0.10
1679,11720.0,2-undecyl-1-ethylamino imidazoline (2UEI),CCCCCCCCCCC/C1=N/CCN1CCN,35.0,313K,80mg/L,0.8354,313.0,80mg/L,0.08
1680,11721.0,2-undecyl-1-ethylamino imidazoline (2UEI),CCCCCCCCCCC/C1=N/CCN1CCN,35.0,323K,80mg/L,0.8461,323.0,80mg/L,0.08


In [31]:
#转化为标准的科学计数法
def fix_scientific_notation(concentration_str):
    # 使用正则表达式匹配科学计数法的模式：数字 * 10 ± 指数 单位
    pattern1 = r"([0-9.]+)\*10([-+]?[0-9]+)([a-zA-Z]+)?"
    pattern2 = r"10(-[0-9]+)([a-zA-Z]+)?"
    match1 = re.match(pattern1, concentration_str.strip())
    match2 = re.match(pattern2, concentration_str.strip())
    if match1:
        number, exponent, unit = match1.groups()
        # 将其转换为标准的科学计数法形式
        if number is None:
            number = 1
        decimal_value = float(number) * (10 ** int(exponent))
        decimal_value = f"{decimal_value:.10f}"
        fixed_notation = f"{decimal_value}{unit or ''}".strip()
        return fixed_notation
    elif match2:
        exponent, unit = match2.groups()
        decimal_value = (10 ** int(exponent))
        decimal_value = f"{decimal_value:.10f}"
        fixed_notation = f"{decimal_value}{unit or ''}".strip()
        print(match2, decimal_value)
        return fixed_notation
    else:
        # 如果不是科学计数法，则返回原字符串
        return concentration_str

In [32]:
# 假设已经读取了CSV文件到df DataFrame中
# df = pd.read_csv('solution_properties.csv', encoding='utf-8')

# 应用函数到'Concentration'列
df['Fixed_Concentration'] = df['Concentration'].apply(fix_scientific_notation)
df.head(898)

<re.Match object; span=(0, 5), match='10-4M'> 0.0001000000
<re.Match object; span=(0, 5), match='10-3M'> 0.0010000000
<re.Match object; span=(0, 5), match='10-4M'> 0.0001000000
<re.Match object; span=(0, 5), match='10-3M'> 0.0010000000


,ID,NAME,SMILES,NaClg/L,Temp,Concentration,IE,TempK,Fixed_Concentration
0,10001.0,2-mercaptopyrimidine,S=c1nccc[nH]1,10.0,313K,20PPM,0.9124,313.0,20PPM
1,10002.0,2-mercaptopyrimidine,S=c1nccc[nH]1,50.0,313K,20PPM,0.8556,313.0,20PPM
2,10003.0,2-mercaptopyrimidine,S=c1nccc[nH]1,100.0,313K,20PPM,0.8331,313.0,20PPM
3,10004.0,2-mercaptopyrimidine,S=c1nccc[nH]1,150.0,313K,20PPM,0.8173,313.0,20PPM
4,10005.0,2-mercaptopyrimidine,S=c1nccc[nH]1,200.0,313K,20PPM,0.7629,313.0,20PPM
...,...,...,...,...,...,...,...,...,...
893,10911.0,Monoethylene glycol,C=C(c1ccccc1)C5(c2ccccc2)/C(c3ccccc3)=C(c4cccc...,100.0,323K,70wt%,0.5630,323.0,70wt%
894,10912.0,Monoethylene glycol,C=C(c1ccccc1)C5(c2ccccc2)/C(c3ccccc3)=C(c4cccc...,100.0,323K,90wt%,0.8200,323.0,90wt%
895,10913.0,Thiourea (TU),NC(N)=S,30.0,333K,10mg/L,0.7980,333.0,10mg/L
896,10914.0,berberine（BB）,COc5ccc4CC3c2cc1OCOc1cc2CCN3Cc4c5OC,30.0,333K,10mg/L,0.0180,333.0,10mg/L


In [33]:
# 定义一个函数进行单位转化，把浓度统一为g/L
def convert_concentration(row):
    try:
        mol = Chem.MolFromSmiles(row['SMILES'])
        molar_mass = Descriptors.MolWt(mol) 
    except:
        molar_mass = 1
    concentration_str = row['Fixed_Concentration']
    # 使用正则表达式匹配数字和单位
    match = re.match(r"([0-9.]+)\s*([a-zA-Z/%]+)", concentration_str)
    if not match:
        raise ValueError(f"无法解析浓度: {concentration_str},row is {row['ID']}")
    
    value, unit = match.groups()
    value = float(value)

    # 根据单位转换为M
    conversion_factors = {
        'PPM': lambda x: x / 1000, # 假设1 PPM = 1 mg/L = 0.001 g/L
        'mM': lambda x: x / 1000 * molar_mass,
        'M': lambda x: x * molar_mass,
        'muM': lambda x: x / 1000000 * molar_mass,
        'g/L': lambda x: x,
        'mg/L': lambda x: x / 1000,
        'wt%': lambda x: x * 10,
        'mL/L': lambda x: x / 1000
    }

    try:
        factor = conversion_factors[unit]
    except KeyError:
        raise ValueError(f"未知单位: {unit},row is {row['ID']}")

    return factor(value)


In [34]:
# 假设'Concentration'列包含浓度字符串，应用函数
df['Concentrationg/L'] = df.apply(convert_concentration, axis=1)

df.head(898)

,ID,NAME,SMILES,NaClg/L,Temp,Concentration,IE,TempK,Fixed_Concentration,Concentrationg/L
0,10001.0,2-mercaptopyrimidine,S=c1nccc[nH]1,10.0,313K,20PPM,0.9124,313.0,20PPM,0.02
1,10002.0,2-mercaptopyrimidine,S=c1nccc[nH]1,50.0,313K,20PPM,0.8556,313.0,20PPM,0.02
2,10003.0,2-mercaptopyrimidine,S=c1nccc[nH]1,100.0,313K,20PPM,0.8331,313.0,20PPM,0.02
3,10004.0,2-mercaptopyrimidine,S=c1nccc[nH]1,150.0,313K,20PPM,0.8173,313.0,20PPM,0.02
4,10005.0,2-mercaptopyrimidine,S=c1nccc[nH]1,200.0,313K,20PPM,0.7629,313.0,20PPM,0.02
...,...,...,...,...,...,...,...,...,...,...
893,10911.0,Monoethylene glycol,C=C(c1ccccc1)C5(c2ccccc2)/C(c3ccccc3)=C(c4cccc...,100.0,323K,70wt%,0.5630,323.0,70wt%,700.00
894,10912.0,Monoethylene glycol,C=C(c1ccccc1)C5(c2ccccc2)/C(c3ccccc3)=C(c4cccc...,100.0,323K,90wt%,0.8200,323.0,90wt%,900.00
895,10913.0,Thiourea (TU),NC(N)=S,30.0,333K,10mg/L,0.7980,333.0,10mg/L,0.01
896,10914.0,berberine（BB）,COc5ccc4CC3c2cc1OCOc1cc2CCN3Cc4c5OC,30.0,333K,10mg/L,0.0180,333.0,10mg/L,0.01


In [44]:
# 保存修改后的DataFrame到新的CSV文件
if 'Fixed_Concentration' in df.columns:
    df = df.drop('Fixed_Concentration', axis=1)
    df = df.drop('Concentration', axis=1)
    df = df.drop('Temp', axis=1)
    df = df.drop('NAME', axis=1)
df.to_csv('solution_properties_converted.csv', index=False, encoding='utf-8')

print("浓度转换完成并已保存到新的CSV文件中。")

浓度转换完成并已保存到新的CSV文件中。


In [80]:
# 将分子信息和其他特征值分开存储
df_2 = df.drop('NaClg/L', axis=1).drop('Concentrationg/L', axis=1).drop('TempK', axis=1)
df_3 = df.drop('SMILES', axis=1).drop('IE', axis=1)
df_3.to_csv('extra_data.csv', index=False, encoding='utf-8')
df_2.to_csv('molecules.csv', index=False, encoding='utf-8')

In [76]:
#检查非数字列
non_numeric_variables = df.select_dtypes(include=['object'])
non_numeric_variables

,SMILES
0,S=c1nccc[nH]1
1,S=c1nccc[nH]1
2,S=c1nccc[nH]1
3,S=c1nccc[nH]1
4,S=c1nccc[nH]1
...,...
1677,CCCCCCCCCCC/C1=N/CCN1CCN
1678,CCCCCCCCCCC/C1=N/CCN1CCN
1679,CCCCCCCCCCC/C1=N/CCN1CCN
1680,CCCCCCCCCCC/C1=N/CCN1CCN


In [67]:
# 只保留氯化钠浓度在25-50之间的
df2 = df[df['NaClg/L']>=25]
df2 = df2[df2['NaClg/L']<=50]

In [11]:
# 正弦函数对环境条件编码
def sine_encode(value, dim=16):
    encoded = []
    for i in range(1, dim + 1):
        encoded.append(np.sin(i * value))  # 使用正弦函数编码
    return encoded

In [68]:
# 对每一列数据进行编码
encoded_columns = []
for column in ['TempK','Concentrationg/L']:
    # 对该列的每个元素进行编码，并将结果放入新的列中
    encoded_column = df2[column].apply(lambda x: sine_encode(x))
    encoded_columns.append((column, encoded_column))

# 将编码后的列合并到原始数据框中
for column, encoded_column in encoded_columns:
    # 生成编码后的列名，并且正确命名为 `Temp1`, `Temp2`, ..., `Temp16`等
    encoded_df = pd.DataFrame(encoded_column.tolist(), columns=[f'{column}{i+1}' for i in range(16)],index = df2.index)
    df2 = pd.concat([df2, encoded_df], axis=1)

In [69]:
df2 = df2.drop()

,ID,SMILES,IE,TempK,Concentrationg/L,TempK1,TempK2,TempK3,TempK4,TempK5,...,Concentrationg/L7,Concentrationg/L8,Concentrationg/L9,Concentrationg/L10,Concentrationg/L11,Concentrationg/L12,Concentrationg/L13,Concentrationg/L14,Concentrationg/L15,Concentrationg/L16
1,10002.0,S=c1nccc[nH]1,0.8556,313.0,0.020000,-0.916509,-0.733231,0.329905,0.997164,0.467852,...,0.139543,0.159318,0.179030,0.198669,0.218230,0.237703,0.257081,0.276356,0.295520,0.314567
14,10015.0,Oc3ccc(/N=N\c2ccc1ccccc1n2)cc3O,0.3980,323.0,0.002653,0.551402,-0.920002,0.983604,-0.721121,0.219572,...,0.018568,0.021220,0.023872,0.026524,0.029176,0.031827,0.034479,0.037130,0.039780,0.042431
15,10016.0,Oc3ccc(/N=N\c2ccc1ccccc1n2)cc3O,0.5890,323.0,0.006632,0.551402,-0.920002,0.983604,-0.721121,0.219572,...,0.046406,0.053030,0.059651,0.066269,0.072885,0.079498,0.086107,0.092712,0.099313,0.105910
16,10017.0,Oc3ccc(/N=N\c2ccc1ccccc1n2)cc3O,0.8260,323.0,0.013264,0.551402,-0.920002,0.983604,-0.721121,0.219572,...,0.092712,0.105910,0.119089,0.132247,0.145383,0.158492,0.171574,0.184625,0.197644,0.210628
17,10018.0,Oc3ccc(/N=N\c2ccc1ccccc1n2)cc3O,0.9140,323.0,0.026527,0.551402,-0.920002,0.983604,-0.721121,0.219572,...,0.184625,0.210628,0.236483,0.262172,0.287676,0.312977,0.338059,0.362902,0.387491,0.411806
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1677,11718.0,CCCCCCCCCCC/C1=N/CCN1CCN,0.7523,298.0,0.080000,0.436138,-0.784943,0.976571,-0.972651,0.773967,...,0.531186,0.597195,0.659385,0.717356,0.770739,0.819192,0.862404,0.900100,0.932039,0.958016
1678,11719.0,CCCCCCCCCCC/C1=N/CCN1CCN,0.8767,298.0,0.100000,0.436138,-0.784943,0.976571,-0.972651,0.773967,...,0.644218,0.717356,0.783327,0.841471,0.891207,0.932039,0.963558,0.985450,0.997495,0.999574
1679,11720.0,CCCCCCCCCCC/C1=N/CCN1CCN,0.8354,313.0,0.080000,-0.916509,-0.733231,0.329905,0.997164,0.467852,...,0.531186,0.597195,0.659385,0.717356,0.770739,0.819192,0.862404,0.900100,0.932039,0.958016
1680,11721.0,CCCCCCCCCCC/C1=N/CCN1CCN,0.8461,323.0,0.080000,0.551402,-0.920002,0.983604,-0.721121,0.219572,...,0.531186,0.597195,0.659385,0.717356,0.770739,0.819192,0.862404,0.900100,0.932039,0.958016


In [70]:
#删除原有列
df2 = df2.drop('Concentrationg/L', axis=1).drop('TempK', axis=1).drop('NaClg/L', axis=1)
df2

,ID,SMILES,IE,TempK1,TempK2,TempK3,TempK4,TempK5,TempK6,TempK7,...,Concentrationg/L7,Concentrationg/L8,Concentrationg/L9,Concentrationg/L10,Concentrationg/L11,Concentrationg/L12,Concentrationg/L13,Concentrationg/L14,Concentrationg/L15,Concentrationg/L16
1,10002.0,S=c1nccc[nH]1,0.8556,-0.916509,-0.733231,0.329905,0.997164,0.467852,-0.622871,-0.966164,...,0.139543,0.159318,0.179030,0.198669,0.218230,0.237703,0.257081,0.276356,0.295520,0.314567
14,10015.0,Oc3ccc(/N=N\c2ccc1ccccc1n2)cc3O,0.3980,0.551402,-0.920002,0.983604,-0.721121,0.219572,0.354769,-0.811498,...,0.018568,0.021220,0.023872,0.026524,0.029176,0.031827,0.034479,0.037130,0.039780,0.042431
15,10016.0,Oc3ccc(/N=N\c2ccc1ccccc1n2)cc3O,0.5890,0.551402,-0.920002,0.983604,-0.721121,0.219572,0.354769,-0.811498,...,0.046406,0.053030,0.059651,0.066269,0.072885,0.079498,0.086107,0.092712,0.099313,0.105910
16,10017.0,Oc3ccc(/N=N\c2ccc1ccccc1n2)cc3O,0.8260,0.551402,-0.920002,0.983604,-0.721121,0.219572,0.354769,-0.811498,...,0.092712,0.105910,0.119089,0.132247,0.145383,0.158492,0.171574,0.184625,0.197644,0.210628
17,10018.0,Oc3ccc(/N=N\c2ccc1ccccc1n2)cc3O,0.9140,0.551402,-0.920002,0.983604,-0.721121,0.219572,0.354769,-0.811498,...,0.184625,0.210628,0.236483,0.262172,0.287676,0.312977,0.338059,0.362902,0.387491,0.411806
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1677,11718.0,CCCCCCCCCCC/C1=N/CCN1CCN,0.7523,0.436138,-0.784943,0.976571,-0.972651,0.773967,-0.420304,-0.017521,...,0.531186,0.597195,0.659385,0.717356,0.770739,0.819192,0.862404,0.900100,0.932039,0.958016
1678,11719.0,CCCCCCCCCCC/C1=N/CCN1CCN,0.8767,0.436138,-0.784943,0.976571,-0.972651,0.773967,-0.420304,-0.017521,...,0.644218,0.717356,0.783327,0.841471,0.891207,0.932039,0.963558,0.985450,0.997495,0.999574
1679,11720.0,CCCCCCCCCCC/C1=N/CCN1CCN,0.8354,-0.916509,-0.733231,0.329905,0.997164,0.467852,-0.622871,-0.966164,...,0.531186,0.597195,0.659385,0.717356,0.770739,0.819192,0.862404,0.900100,0.932039,0.958016
1680,11721.0,CCCCCCCCCCC/C1=N/CCN1CCN,0.8461,0.551402,-0.920002,0.983604,-0.721121,0.219572,0.354769,-0.811498,...,0.531186,0.597195,0.659385,0.717356,0.770739,0.819192,0.862404,0.900100,0.932039,0.958016


In [ ]:
# 计算分子描述符
def CalcMolDescriptors(mol):
    if mol is None:
        return [None] * len(Descriptors._descList)  # 如果分子为空，返回None
    return [desc[1](mol) for desc in Descriptors._descList]  # 计算所有分子描述符

In [92]:
mols = [Chem.MolFromSmiles(smiles) for smiles in df2['SMILES']]

# 4. 使用列表推导式计算每个分子的208个特征
descriptors = [Chem.Descriptors.CalcMolDescriptors(mol) for mol in mols]

# 5. 将描述符结果保存到DataFrame
columns = [desc[0] for desc in Descriptors._descList]  # 获取所有描述符的名字
df_descriptors = pd.DataFrame(descriptors, columns=columns)
df_descriptors.fillna(0, inplace=True)  

df_descriptors

[17:41:24] DEPRECATION WARNING: please use MorganGenerator
[17:41:24] DEPRECATION WARNING: please use MorganGenerator
[17:41:24] DEPRECATION WARNING: please use MorganGenerator
[17:41:24] DEPRECATION WARNING: please use MorganGenerator
[17:41:24] DEPRECATION WARNING: please use MorganGenerator
[17:41:24] DEPRECATION WARNING: please use MorganGenerator
[17:41:24] DEPRECATION WARNING: please use MorganGenerator
[17:41:24] DEPRECATION WARNING: please use MorganGenerator
[17:41:24] DEPRECATION WARNING: please use MorganGenerator
[17:41:24] DEPRECATION WARNING: please use MorganGenerator
[17:41:24] DEPRECATION WARNING: please use MorganGenerator
[17:41:24] DEPRECATION WARNING: please use MorganGenerator
[17:41:24] DEPRECATION WARNING: please use MorganGenerator
[17:41:24] DEPRECATION WARNING: please use MorganGenerator
[17:41:24] DEPRECATION WARNING: please use MorganGenerator
[17:41:24] DEPRECATION WARNING: please use MorganGenerator
[17:41:24] DEPRECATION WARNING: please use MorganGenerat

[17:41:25] DEPRECATION WARNING: please use MorganGenerator
[17:41:25] DEPRECATION WARNING: please use MorganGenerator
[17:41:25] DEPRECATION WARNING: please use MorganGenerator
[17:41:25] DEPRECATION WARNING: please use MorganGenerator
[17:41:25] DEPRECATION WARNING: please use MorganGenerator
[17:41:25] DEPRECATION WARNING: please use MorganGenerator
[17:41:25] DEPRECATION WARNING: please use MorganGenerator
[17:41:25] DEPRECATION WARNING: please use MorganGenerator
[17:41:25] DEPRECATION WARNING: please use MorganGenerator
[17:41:25] DEPRECATION WARNING: please use MorganGenerator
[17:41:25] DEPRECATION WARNING: please use MorganGenerator
[17:41:25] DEPRECATION WARNING: please use MorganGenerator
[17:41:25] DEPRECATION WARNING: please use MorganGenerator
[17:41:25] DEPRECATION WARNING: please use MorganGenerator
[17:41:25] DEPRECATION WARNING: please use MorganGenerator
[17:41:25] DEPRECATION WARNING: please use MorganGenerator
[17:41:25] DEPRECATION WARNING: please use MorganGenerat

[17:41:25] DEPRECATION WARNING: please use MorganGenerator
[17:41:25] DEPRECATION WARNING: please use MorganGenerator
[17:41:25] DEPRECATION WARNING: please use MorganGenerator
[17:41:25] DEPRECATION WARNING: please use MorganGenerator
[17:41:25] DEPRECATION WARNING: please use MorganGenerator
[17:41:25] DEPRECATION WARNING: please use MorganGenerator
[17:41:26] DEPRECATION WARNING: please use MorganGenerator
[17:41:26] DEPRECATION WARNING: please use MorganGenerator
[17:41:26] DEPRECATION WARNING: please use MorganGenerator
[17:41:26] DEPRECATION WARNING: please use MorganGenerator
[17:41:26] DEPRECATION WARNING: please use MorganGenerator
[17:41:26] DEPRECATION WARNING: please use MorganGenerator
[17:41:26] DEPRECATION WARNING: please use MorganGenerator
[17:41:26] DEPRECATION WARNING: please use MorganGenerator
[17:41:26] DEPRECATION WARNING: please use MorganGenerator
[17:41:26] DEPRECATION WARNING: please use MorganGenerator
[17:41:26] DEPRECATION WARNING: please use MorganGenerat

[17:41:26] DEPRECATION WARNING: please use MorganGenerator
[17:41:26] DEPRECATION WARNING: please use MorganGenerator
[17:41:26] DEPRECATION WARNING: please use MorganGenerator
[17:41:26] DEPRECATION WARNING: please use MorganGenerator
[17:41:26] DEPRECATION WARNING: please use MorganGenerator
[17:41:26] DEPRECATION WARNING: please use MorganGenerator
[17:41:26] DEPRECATION WARNING: please use MorganGenerator
[17:41:26] DEPRECATION WARNING: please use MorganGenerator
[17:41:26] DEPRECATION WARNING: please use MorganGenerator
[17:41:26] DEPRECATION WARNING: please use MorganGenerator
[17:41:26] DEPRECATION WARNING: please use MorganGenerator
[17:41:26] DEPRECATION WARNING: please use MorganGenerator
[17:41:26] DEPRECATION WARNING: please use MorganGenerator
[17:41:26] DEPRECATION WARNING: please use MorganGenerator
[17:41:26] DEPRECATION WARNING: please use MorganGenerator
[17:41:26] DEPRECATION WARNING: please use MorganGenerator
[17:41:26] DEPRECATION WARNING: please use MorganGenerat

[17:41:27] DEPRECATION WARNING: please use MorganGenerator
[17:41:27] DEPRECATION WARNING: please use MorganGenerator
[17:41:27] DEPRECATION WARNING: please use MorganGenerator
[17:41:27] DEPRECATION WARNING: please use MorganGenerator
[17:41:27] DEPRECATION WARNING: please use MorganGenerator
[17:41:27] DEPRECATION WARNING: please use MorganGenerator
[17:41:27] DEPRECATION WARNING: please use MorganGenerator
[17:41:27] DEPRECATION WARNING: please use MorganGenerator
[17:41:27] DEPRECATION WARNING: please use MorganGenerator
[17:41:27] DEPRECATION WARNING: please use MorganGenerator
[17:41:27] DEPRECATION WARNING: please use MorganGenerator
[17:41:27] DEPRECATION WARNING: please use MorganGenerator
[17:41:27] DEPRECATION WARNING: please use MorganGenerator
[17:41:27] DEPRECATION WARNING: please use MorganGenerator
[17:41:27] DEPRECATION WARNING: please use MorganGenerator
[17:41:27] DEPRECATION WARNING: please use MorganGenerator
[17:41:27] DEPRECATION WARNING: please use MorganGenerat

...

[17:41:34] DEPRECATION WARNING: please use MorganGenerator
[17:41:34] DEPRECATION WARNING: please use MorganGenerator
[17:41:34] DEPRECATION WARNING: please use MorganGenerator
[17:41:34] DEPRECATION WARNING: please use MorganGenerator
[17:41:34] DEPRECATION WARNING: please use MorganGenerator
[17:41:34] DEPRECATION WARNING: please use MorganGenerator
[17:41:34] DEPRECATION WARNING: please use MorganGenerator
[17:41:34] DEPRECATION WARNING: please use MorganGenerator
[17:41:34] DEPRECATION WARNING: please use MorganGenerator
[17:41:34] DEPRECATION WARNING: please use MorganGenerator
[17:41:34] DEPRECATION WARNING: please use MorganGenerator
[17:41:34] DEPRECATION WARNING: please use MorganGenerator
[17:41:34] DEPRECATION WARNING: please use MorganGenerator
[17:41:34] DEPRECATION WARNING: please use MorganGenerator
[17:41:34] DEPRECATION WARNING: please use MorganGenerator
[17:41:34] DEPRECATION WARNING: please use MorganGenerator
[17:41:34] DEPRECATION WARNING: please use MorganGenerat

[17:41:34] DEPRECATION WARNING: please use MorganGenerator
[17:41:34] DEPRECATION WARNING: please use MorganGenerator
[17:41:34] DEPRECATION WARNING: please use MorganGenerator
[17:41:34] DEPRECATION WARNING: please use MorganGenerator
[17:41:34] DEPRECATION WARNING: please use MorganGenerator
[17:41:34] DEPRECATION WARNING: please use MorganGenerator
[17:41:34] DEPRECATION WARNING: please use MorganGenerator
[17:41:34] DEPRECATION WARNING: please use MorganGenerator
[17:41:34] DEPRECATION WARNING: please use MorganGenerator
[17:41:35] DEPRECATION WARNING: please use MorganGenerator
[17:41:35] DEPRECATION WARNING: please use MorganGenerator
[17:41:35] DEPRECATION WARNING: please use MorganGenerator
[17:41:35] DEPRECATION WARNING: please use MorganGenerator
[17:41:35] DEPRECATION WARNING: please use MorganGenerator
[17:41:35] DEPRECATION WARNING: please use MorganGenerator
[17:41:35] DEPRECATION WARNING: please use MorganGenerator
[17:41:35] DEPRECATION WARNING: please use MorganGenerat

[17:41:35] DEPRECATION WARNING: please use MorganGenerator
[17:41:35] DEPRECATION WARNING: please use MorganGenerator
[17:41:35] DEPRECATION WARNING: please use MorganGenerator
[17:41:35] DEPRECATION WARNING: please use MorganGenerator
[17:41:35] DEPRECATION WARNING: please use MorganGenerator
[17:41:35] DEPRECATION WARNING: please use MorganGenerator
[17:41:35] DEPRECATION WARNING: please use MorganGenerator
[17:41:35] DEPRECATION WARNING: please use MorganGenerator
[17:41:35] DEPRECATION WARNING: please use MorganGenerator
[17:41:35] DEPRECATION WARNING: please use MorganGenerator
[17:41:35] DEPRECATION WARNING: please use MorganGenerator
[17:41:35] DEPRECATION WARNING: please use MorganGenerator
[17:41:35] DEPRECATION WARNING: please use MorganGenerator
[17:41:35] DEPRECATION WARNING: please use MorganGenerator
[17:41:35] DEPRECATION WARNING: please use MorganGenerator
[17:41:35] DEPRECATION WARNING: please use MorganGenerator
[17:41:35] DEPRECATION WARNING: please use MorganGenerat

[17:41:36] DEPRECATION WARNING: please use MorganGenerator
[17:41:36] DEPRECATION WARNING: please use MorganGenerator
[17:41:36] DEPRECATION WARNING: please use MorganGenerator
[17:41:36] DEPRECATION WARNING: please use MorganGenerator
[17:41:36] DEPRECATION WARNING: please use MorganGenerator
[17:41:36] DEPRECATION WARNING: please use MorganGenerator
[17:41:36] DEPRECATION WARNING: please use MorganGenerator
[17:41:36] DEPRECATION WARNING: please use MorganGenerator
[17:41:36] DEPRECATION WARNING: please use MorganGenerator
[17:41:36] DEPRECATION WARNING: please use MorganGenerator
[17:41:36] DEPRECATION WARNING: please use MorganGenerator
[17:41:36] DEPRECATION WARNING: please use MorganGenerator
[17:41:36] DEPRECATION WARNING: please use MorganGenerator
[17:41:36] DEPRECATION WARNING: please use MorganGenerator
[17:41:36] DEPRECATION WARNING: please use MorganGenerator
[17:41:36] DEPRECATION WARNING: please use MorganGenerator
[17:41:36] DEPRECATION WARNING: please use MorganGenerat

,MaxAbsEStateIndex,MaxEStateIndex,MinAbsEStateIndex,MinEStateIndex,qed,SPS,MolWt,HeavyAtomMolWt,ExactMolWt,NumValenceElectrons,...,fr_sulfide,fr_sulfonamd,fr_sulfone,fr_term_acetylene,fr_tetrazole,fr_thiazole,fr_thiocyan,fr_thiophene,fr_unbrch_alkane,fr_urea
0,4.645370,4.645370,0.530093,0.530093,0.509669,8.571429,112.157,108.125,112.009519,36,...,0,0,0,0,0,0,0,0,0,0
1,9.371344,9.371344,0.187241,-0.228777,0.541365,11.200000,265.272,254.184,265.085127,98,...,0,0,0,0,0,0,0,0,0,0
2,9.371344,9.371344,0.187241,-0.228777,0.541365,11.200000,265.272,254.184,265.085127,98,...,0,0,0,0,0,0,0,0,0,0
3,9.371344,9.371344,0.187241,-0.228777,0.541365,11.200000,265.272,254.184,265.085127,98,...,0,0,0,0,0,0,0,0,0,0
4,9.371344,9.371344,0.187241,-0.228777,0.541365,11.200000,265.272,254.184,265.085127,98,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1009,5.618219,5.618219,0.746806,0.746806,0.549123,15.052632,267.461,234.197,267.267448,112,...,0,0,0,0,0,0,0,0,8,0
1010,5.618219,5.618219,0.746806,0.746806,0.549123,15.052632,267.461,234.197,267.267448,112,...,0,0,0,0,0,0,0,0,8,0
1011,5.618219,5.618219,0.746806,0.746806,0.549123,15.052632,267.461,234.197,267.267448,112,...,0,0,0,0,0,0,0,0,8,0
1012,5.618219,5.618219,0.746806,0.746806,0.549123,15.052632,267.461,234.197,267.267448,112,...,0,0,0,0,0,0,0,0,8,0


In [84]:
df3

,index,SMILES,IE,TempK1,TempK2,TempK3,TempK4,TempK5,TempK6,TempK7,...,fr_sulfide,fr_sulfonamd,fr_sulfone,fr_term_acetylene,fr_tetrazole,fr_thiazole,fr_thiocyan,fr_thiophene,fr_unbrch_alkane,fr_urea
0,1,S=c1nccc[nH]1,0.8556,-0.916509,-0.733231,0.329905,0.997164,0.467852,-0.622871,-0.966164,...,0,0,0,0,0,0,0,0,0,0
1,14,Oc3ccc(/N=N\c2ccc1ccccc1n2)cc3O,0.3980,0.551402,-0.920002,0.983604,-0.721121,0.219572,0.354769,-0.811498,...,0,0,0,0,0,0,0,0,0,0
2,15,Oc3ccc(/N=N\c2ccc1ccccc1n2)cc3O,0.5890,0.551402,-0.920002,0.983604,-0.721121,0.219572,0.354769,-0.811498,...,0,0,0,0,0,0,0,0,0,0
3,16,Oc3ccc(/N=N\c2ccc1ccccc1n2)cc3O,0.8260,0.551402,-0.920002,0.983604,-0.721121,0.219572,0.354769,-0.811498,...,0,0,0,0,0,0,0,0,0,0
4,17,Oc3ccc(/N=N\c2ccc1ccccc1n2)cc3O,0.9140,0.551402,-0.920002,0.983604,-0.721121,0.219572,0.354769,-0.811498,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1009,1677,CCCCCCCCCCC/C1=N/CCN1CCN,0.7523,0.436138,-0.784943,0.976571,-0.972651,0.773967,-0.420304,-0.017521,...,0,0,0,0,0,0,0,0,8,0
1010,1678,CCCCCCCCCCC/C1=N/CCN1CCN,0.8767,0.436138,-0.784943,0.976571,-0.972651,0.773967,-0.420304,-0.017521,...,0,0,0,0,0,0,0,0,8,0
1011,1679,CCCCCCCCCCC/C1=N/CCN1CCN,0.8354,-0.916509,-0.733231,0.329905,0.997164,0.467852,-0.622871,-0.966164,...,0,0,0,0,0,0,0,0,8,0
1012,1680,CCCCCCCCCCC/C1=N/CCN1CCN,0.8461,0.551402,-0.920002,0.983604,-0.721121,0.219572,0.354769,-0.811498,...,0,0,0,0,0,0,0,0,8,0


In [93]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaled_array = scaler.fit_transform(df_descriptors)
df_descriptors = pd.DataFrame(scaled_array, columns=df_descriptors.columns, index=df_descriptors.index)
df_descriptors

,MaxAbsEStateIndex,MaxEStateIndex,MinAbsEStateIndex,MinEStateIndex,qed,SPS,MolWt,HeavyAtomMolWt,ExactMolWt,NumValenceElectrons,...,fr_sulfide,fr_sulfonamd,fr_sulfone,fr_term_acetylene,fr_tetrazole,fr_thiazole,fr_thiocyan,fr_thiophene,fr_unbrch_alkane,fr_urea
0,0.175497,0.175497,0.465802,0.910212,0.579517,0.049315,0.022173,0.022856,0.022165,0.018018,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
1,0.512308,0.512308,0.164532,0.798131,0.616474,0.099726,0.116393,0.115492,0.116414,0.111111,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
2,0.512308,0.512308,0.164532,0.798131,0.616474,0.099726,0.116393,0.115492,0.116414,0.111111,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
3,0.512308,0.512308,0.164532,0.798131,0.616474,0.099726,0.116393,0.115492,0.116414,0.111111,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
4,0.512308,0.512308,0.164532,0.798131,0.616474,0.099726,0.116393,0.115492,0.116414,0.111111,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1009,0.244830,0.244830,0.656232,0.942220,0.625521,0.173612,0.117740,0.102815,0.117758,0.132132,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.181818,0.0
1010,0.244830,0.244830,0.656232,0.942220,0.625521,0.173612,0.117740,0.102815,0.117758,0.132132,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.181818,0.0
1011,0.244830,0.244830,0.656232,0.942220,0.625521,0.173612,0.117740,0.102815,0.117758,0.132132,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.181818,0.0
1012,0.244830,0.244830,0.656232,0.942220,0.625521,0.173612,0.117740,0.102815,0.117758,0.132132,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.181818,0.0


In [94]:
#合并到df3
df3 = df2.reset_index()
df3 = pd.concat([df3, df_descriptors], axis=1)
df3 = df3.drop('ID', axis=1)

In [95]:
df3.to_csv('input_zz.csv', index=False, encoding='utf-8')

print("浓度转换完成并已保存到新的CSV文件中。")

浓度转换完成并已保存到新的CSV文件中。


In [ ]:
# 分子信息获取

df['mol'] = df['SMILES'].apply(Chem.MolFromSmiles)
df['HA'] = df['mol'].apply(Lipinski.NumHAcceptors)
df['HD'] = df['mol'].apply(Lipinski.NumHDonors)
df['RB'] = df['mol'].apply(Lipinski.NumRotatableBonds)
df['Al'] = df['mol'].apply(Lipinski.NumAliphaticRings)
df['Ar'] = df['mol'].apply(Lipinski.NumAromaticRings)
df['F3'] = df['mol'].apply(Lipinski.FractionCSP3)
df['MW'] = df['mol'].apply(Descriptors.ExactMolWt)
df['MLP'] = df['mol'].apply(Descriptors.MolLogP)
df['TPSA'] = df['mol'].apply(Chem.rdMolDescriptors.CalcTPSA)
df['MMR'] = df['mol'].apply(Descriptors.MolMR)

In [89]:
df_descriptors

array([[0.1754973 , 0.1754973 , 0.46580219, ..., 0.        , 0.        ,
        0.        ],
       [0.51230768, 0.51230768, 0.16453179, ..., 0.        , 0.        ,
        0.        ],
       [0.51230768, 0.51230768, 0.16453179, ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.2448302 , 0.2448302 , 0.65623188, ..., 0.        , 0.18181818,
        0.        ],
       [0.2448302 , 0.2448302 , 0.65623188, ..., 0.        , 0.18181818,
        0.        ],
       [0.2448302 , 0.2448302 , 0.65623188, ..., 0.        , 0.18181818,
        0.        ]])